In [1]:
import os
import json

from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [3]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [4]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [13]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [11]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [11]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [12]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [28]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [30]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [32]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [34]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

... to be continued

In [ ]:
get_tables(db_path)

--------------